#### classificador de span

In [17]:
import os
import io
import numpy
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)  
            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message

# função para criar um dataframe a partir de um diretório de emails, classificando como spam ou nao
def dataframeFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})  # adiciona a mensagem e sua classificação
        index.append(filename) 
    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

# adiciona as mensagens de spam ao dataframe
data = pd.concat([data, dataframeFromDirectory("emails/spam", "spam")])

# adiciona as mensagens confiaveis ao dataframe
data = pd.concat([data, dataframeFromDirectory("emails/ham", "ham")])


In [18]:
data.head()

,message,class
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


In [19]:

#objeto CountVectorizer para transformar o texto das mensagens em vetores de contagem de palavras
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values) 

classifier = MultinomialNB()#naive bayes multinomial
targets = data['class'].values  # classes alvo spam ou nao do dataframe
classifier.fit(counts, targets)  # treina o classificador usando os vetores de contagem e as classes


MultinomialNB()

In [21]:

#testa o classificador com exemplos de novas mensagens
examples = ['Free Viagra now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples) 
predictions = classifier.predict(example_counts)  #previsões sobre as novas mensagens
predictions


array(['spam', 'ham'], dtype='<U4')